In [6]:
import pandas as pd
from scipy.io import loadmat
import tables
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns; sns.set()
import glob

In [7]:
"""
Author: Trevor Jordan Grant.
default4: spatial/verbal
"""
# Dictionary is labeled such that task labels in conditions files will have
# more than one multilabeling schema.

# To add more multilabeling schema - include them in the task Dictionary.
# (The sub-dictionary where the task label is the key.)

# default4 = ["VerbalWM", "SpatialWM", "VisualPerceptual", "AuditoryPerceptual"]
# every label in default 4 has discrete values of 'off', 'low', 'high'

# default3 = ["WM", "VisualPerceptual", "AuditoryPerceptual"]
# every label in default 3 has discrete values of 'off', 'low', 'high'

cog_load_label_dict = {
# Mindfulness task labels.
                       "nb": {
                              "default4": ["high", "off", "low", "off"],
                              "default3": ["high", "low", "off"],
                             },
                       "anb": {
                               "default4": ["high", "off", "off", "low"],
                               "default3": ["high", "off", "low"],
                              },
                       "ewm": {
                               "default4": ["low", "off", "high", "off"],
                               "default3": ["low", "high", "off"]
                              },
                        "cr": {
                               "default4": ["off", "off", "off", "off"],
                               "default3": ["off", "off", "off"],
                              },
                        "rt": {
                               "default4": ["off", "off", "low", "off"],
                               "default3": ["off", "low", "off"],
                              },
                        "es": {
                               "default4": ["off", "off", "low", "off"],
                               "default3": ["off", "low", "off"],
                              },
                       "gng": {
                               "default4": ["off", "off", "low", "off"],
                               "default3": ["off", "low", "off"],
                              },
# 7000s (CRA) Task Labels - (Default load lvl).
            "adaptive_words": {
                               "default4": ["off", "off", "low", "off"],
                               "default3": ["off", "low", "off"],
                              },
                   "go_nogo": {
                               "default4": ["off", "off", "low", "off"],
                               "default3": ["off", "low", "off"],
                              },
                     "nback": {
                               "default4": ["high", "off", "low", "off"],
                               "default3": ["high", "low", "off"],
                              },
                    "posner": {
                               "default4": ["off", "off", "low", "off"],
                               "default3": ["off", "low", "off"],
                              },
                 "simple_rt": {
                               "default4": ["off", "off", "low", "off"],
                               "default3": ["off", "low", "off"],
                              },
             "visual_search": {
                               "default4": ["off", "off", "high", "off"],
                               "default3": ["off", "high", "off"],
                              },
                      }


def strings_to_vectors(string_labels, as_list=False):
    """Maps strings in dict to interger values.
    Args:
        string_labels(list): The string label value of load.
        as_list(bool): False, if True, return list instead of np.array()
    Returns:
        labels as np.array()
    """

    maps = {
            "off": 0,
            "low": 1,
            "high": 2,
           }

    if as_list:
        return [maps[label] for label in string_labels]
    return np.array([maps[label] for label in string_labels])


def return_label(task, label_type="default3", as_strings=False):
    """Returns a label from the cog_load_label_dict.
    Args:
        task(str): The task label from the coditions file.
        label_type(string): The label schema used for the model.
        as_strings(bool): False, if True, return string (in list) values instead.
    Returns:
        labels(np.array): Under defaults labels will be returned as interger
        values in a np.array().
    """
    if as_strings:
        return cog_load_label_dict[task][label_type]
    return strings_to_vectors(cog_load_label_dict[task][label_type])

In [8]:
channel_52_mat = {
           1:[0,1],  2:[0,2],  3:[0,3],  4:[0,4],  5:[0,5],  6:[0,6],  7:[0,7],  8:[0,8],  9:[0,9], 10:[0,10], 
11:[1,0], 12:[1,1], 13:[1,2], 14:[1,3], 15:[1,4], 16:[1,5], 17:[1,6], 18:[1,7], 19:[1,8], 20:[1,9], 21:[1,10], 
          22:[2,1], 23:[2,2], 24:[2,3], 25:[2,4], 26:[2,5], 27:[2,6], 28:[2,7], 29:[2,8], 30:[2,9], 31:[2,10], 
32:[3,0], 33:[3,1], 34:[3,2], 35:[3,3], 36:[3,4], 37:[3,5], 38:[3,6], 39:[3,7], 40:[3,8], 41:[3,9], 42:[3,10], 
          43:[4,1], 44:[4,2], 45:[4,3], 46:[4,4], 47:[4,5], 48:[4,6], 49:[4,7], 50:[4,8], 51:[4,9], 52:[4,10]
}

def get_52_mat(data):
    # returns a matrix of size 5x11.
    mat = np.zeros((5, 11))
    for idx, i in enumerate((data)):
        loc = channel_52_mat[idx+1]
        mat[loc[0], loc[1]] = i
    return mat



In [13]:

def collapse_tasks(tasks, min_dur):
    collapsed_tasks = []
    for i in range(1, len(tasks)):
        t1 = tasks[i-1]
        t2 = tasks[i]
        if t1["class"] == t2["class"] and (t1["duration"] < min_dur and t2["duration"] < min_dur):
            # merge and append
            pass
        else:
            # just append
            pass

def read_tasks(condition, data):
    # conditions, data = csv, mat files
    # tuple containing (class, onset, duration, offset, oxy_data, dxy_data)
    tasks = []
    # read conditions, data
    c_data = pd.read_csv(condition)
    m_data = loadmat(data)
    
    # get oxy, dxy data
    oxyDaya = m_data['nirs_data'][0][0][0]
    dxyData = m_data['nirs_data'][0][0][1]
    
    # iterate through all the tasks here now.
    for idx, key in enumerate(list(c_data.keys())):
        if 'Task' in key:
            # get start and end index of the task
            start = int(c_data[key][0])
            duration = int(c_data[key][1])
            end = start + duration
            
            # visualize heatmap: 
            # sns.heatmap(get_52_mat(oxyDaya[0]))

            oxy_series = oxyDaya[start:end, :]
            dxy_series = dxyData[start:end, :]

            # a 100x5x22 list
            oxy_dxy_series_mat = np.zeros((duration, 5, 22))

            for ts, (oxy_slice, dxy_slice) in enumerate(zip(oxy_series, dxy_series)):
                oxy_slice = get_52_mat(oxy_slice)
                dxy_slice = get_52_mat(dxy_slice)

                oxy_dxy_series_mat[ts] = np.hstack([oxy_slice, dxy_slice])

            tasks.append(
                {
                    "class": c_data[key][2],
                    "onset": start,
                    "end": end,
                    "duration": duration,
                    "data" : oxy_dxy_series_mat
                }
            )
    return tasks 


In [10]:
conditions = glob.glob('../../../data/multilabel/mats/*_conditions*')
data = glob.glob('../../../data/multilabel/mats/*.mat')

In [12]:
time_series_length = 100
multilabel_data = []
task_data = []
"""
default3 labels
[ 
    wm, 
    v, 
    a
]
"""

for idx, (cond, dat) in enumerate(zip(conditions, data)):
    tasks = read_tasks(cond, dat)
    for t in tasks:
        task_data.append(t)

[{'onset': 878, 'data': array([[[ 0.00000000e+00,  1.20481133e+00, -3.16809392e+00, ...,
         -7.06732440e-03,  1.68075806e-02,  2.16485056e-01],
        [ 7.86575905e-01,  2.24510839e+00, -7.50311035e-01, ...,
          4.79651061e-01, -1.92209554e-01,  9.47902678e-01],
        [ 0.00000000e+00,  9.85598296e-01,  1.12488797e+00, ...,
          5.45128566e-01, -2.23756415e-01, -3.39047324e-01],
        [-4.71023814e+00,  1.16601537e+00,  7.06865159e-01, ...,
          1.35355925e-01, -4.07375957e-01, -1.02551506e+00],
        [ 0.00000000e+00, -7.95524432e+00,  7.22207454e-01, ...,
         -1.86943265e-01, -2.29963879e-01, -1.14129159e+00]],

       [[ 0.00000000e+00,  1.20715734e+00, -3.17931148e+00, ...,
         -7.93362501e-03,  3.14563669e-02,  2.24050557e-01],
        [ 7.93235281e-01,  2.24926969e+00, -7.68140112e-01, ...,
          4.78422323e-01, -1.91881473e-01,  9.37654996e-01],
        [ 0.00000000e+00,  1.00276587e+00,  1.12411294e+00, ...,
          5.61712919e-01, -

In [17]:
"""
    write data to disk
"""
for idx, (data, label) in enumerate(multilabel_data[0:601]):
    np.save("C:\\Users\\dhruv\\Development\\git\\thesis_dl-fnirs\\data\\multilabel\\train\\" + str(idx), np.asarray([data, label]))

for idx, (data, label) in enumerate(multilabel_data[601:]):
    np.save("C:\\Users\\dhruv\\Development\\git\\thesis_dl-fnirs\\data\\multilabel\\val\\" + str(idx), np.asarray([data, label]))


In [11]:
train_class_hist

array([[421.,  60., 120., 421., 120.,  60., 541.,  60.,   0.]])

In [12]:
test_class_hist

array([[27.,  4.,  8., 27.,  8.,  4., 35.,  4.,  0.]])